In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Using Vertex AI Matching Engine and Vertex AI Embeddings for Text for StackOverflow Questions 

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/matching_engine/sdk_matching_engine_create_stack_overflow_embeddings_vertex.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/matching_engine/sdk_matching_engine_create_stack_overflow_embeddings_vertex.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
      <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/matching_engine/sdk_matching_engine_create_stack_overflow_embeddings_vertex">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      Open in Vertex AI Workbench
    </a>
  </td>
</table>

**_NOTE_**: This notebook has been tested in the following environment:

* Python version = 3.9

## Overview

This example demonstrates how to encode text as embeddings using the Vertex AI Embeddings for Text service and the StackOverflow dataset. The embeddings are uploaded to the Vertex AI Matching Engine service, which is a high-scale, low-latency solution, for finding similar vectors from a large corpus. Matching Engine is a fully managed offering, further reducing operational overhead. It is built upon [Approximate Nearest Neighbor (ANN) technology](https://ai.googleblog.com/2020/07/announcing-scann-efficient-vector.html) developed by Google Research.

Learn more about [Vertex AI Matching Engine](https://cloud.google.com/vertex-ai/docs/matching-engine/overview) and [Vertex AI Embeddings for Text](https://cloud.google.com/vertex-ai/docs/generative-ai/embeddings/get-text-embeddings).

### Objective

In this notebook, you learn how to encode text as embeddings, create an Approximate Nearest Neighbor (ANN) index, and query against indexes.

This tutorial uses the following Google Cloud ML services:

- `Vertex AI Matching Engine`
- `Vertex AI Embeddings for Text`

The steps performed include:

* Convert a BigQuery dataset to embeddings
* Create an index
* Upload embeddings to the index
* Create an index endpoint
* Deploy the index to the index endpoint
* Perform an online query
* Add metadata to a Redis store

### Dataset

The dataset used for this tutorial is the [StackOverflow dataset](https://console.cloud.google.com/marketplace/product/stack-exchange/stack-overflow).

> Stack Overflow is the largest online community for programmers to learn, share their knowledge, and advance their careers. Updated on a quarterly basis, this BigQuery dataset includes an archive of Stack Overflow content, including posts, votes, tags, and badges. This dataset is updated to mirror the Stack Overflow content on the Internet Archive, and is also available through the Stack Exchange Data Explorer.

### Costs 

This tutorial uses billable components of Google Cloud:

* Vertex AI
* BigQuery
* Cloud Storage

Learn about [Vertex AI pricing](https://cloud.google.com/vertex-ai/pricing), [BigQuery pricing](https://cloud.google.com/bigquery/pricing), and [Cloud Storage pricing](https://cloud.google.com/storage/pricing), 
and use the [Pricing Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

## Installation

Install the latest version of Cloud Storage, BigQuery, and the Vertex AI SDK for Python. Also install the latest version of Redis for low-latency data retrieval.

In [ ]:
# Install the packages
! pip3 install --upgrade google-cloud-aiplatform \
                        google-cloud-storage \
                        'google-cloud-bigquery[pandas]' \
                        redis

### Colab only: Uncomment the following cell to restart the kernel

In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

## Before you begin

### Set up your Google Cloud project

**The following steps are required, regardless of your notebook environment.**

1. [Select or create a Google Cloud project](https://console.cloud.google.com/cloud-resource-manager). When you first create an account, you get a $300 free credit towards your compute/storage costs.

2. [Make sure that billing is enabled for your project](https://cloud.google.com/billing/docs/how-to/modify-project).

3. [Enable the Vertex AI and Redis APIs](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com,redis.googleapis.com).
4. If you are running this notebook locally, you need to install the [Cloud SDK](https://cloud.google.com/sdk).

#### Set your project ID

If you don't know your project ID, try the following:
* Run `gcloud config list`
* Run `gcloud projects list`
* See the support page: [Locate the project ID](https://support.google.com/googleapi/answer/7014113)

In [ ]:
PROJECT_ID = "[YOUR-PROJECT-ID]"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

#### Region

You can also change the `REGION` variable used by Vertex AI. Learn more about [Vertex AI regions](https://cloud.google.com/vertex-ai/docs/general/locations).

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

### Authenticate your Google Cloud account

Depending on your Jupyter environment, you may have to manually authenticate. Follow the relevant instructions below.

**1. Vertex AI Workbench**
* Do nothing as you are already authenticated.

**2. Local JupyterLab instance, uncomment and run:**

In [ ]:
# ! gcloud auth login

**3. Colab, uncomment and run:**

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

**4. Service account or other**
* See how to grant Cloud Storage permissions to your service account at https://cloud.google.com/storage/docs/gsutil/commands/iam#ch-examples.

* Authentication: Rerun the `gcloud auth login` command in the Vertex AI Workbench notebook terminal when you are logged out and need the credential again.

### Create a Cloud Storage bucket

Create a storage bucket to store intermediate artifacts such as datasets.

In [ ]:
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [ ]:
! gsutil mb -l {REGION} -p {PROJECT_ID} {BUCKET_URI}

### Import libraries

In [ ]:
import os
import gc
import json
import math
import time
import redis
import random
import tempfile
import functools
import pandas as pd
import numpy as np
from pathlib import Path
from tqdm.auto import tqdm
from google.cloud import bigquery
from google.cloud import aiplatform
from concurrent.futures import ThreadPoolExecutor
from typing import Any, Generator, List, Tuple, Optional
from vertexai.preview.language_models import TextEmbeddingModel

### Initialize Vertex AI SDK for Python

Initialize the Vertex AI SDK for Python for your project.

In [ ]:
aiplatform.init(project=PROJECT_ID, 
                location=REGION, 
                staging_bucket=BUCKET_URI)

## Prepare the data

You use the [Stack Overflow dataset](https://console.cloud.google.com/marketplace/product/stack-exchange/stack-overflow) of question and answers hosted on BigQuery.

> This public dataset is hosted in Google BigQuery and is included in BigQuery's 1TB/mo of free tier processing. This means that each user receives 1TB of free BigQuery processing every month, which can be used to run queries on this public dataset.

The BigQuery table is too large to fit into memory, so you need to write a generator called `query_bigquery_chunks` to yield chunks of the dataframe for processing. Additionally, an extra column `title_with_body` is added, which is a concatenation of the question title and body.

In [ ]:
client = bigquery.Client(project=PROJECT_ID)
QUERY_TEMPLATE = """
        SELECT distinct q.id, q.title, q.body
        FROM (SELECT * FROM `bigquery-public-data.stackoverflow.posts_questions` where Score>0 ORDER BY View_Count desc) AS q 
        LIMIT {limit} OFFSET {offset};
        """


def query_bigquery_chunks(
    max_rows: int, rows_per_chunk: int, start_chunk: int = 0
) -> Generator[pd.DataFrame, Any, None]:
    for offset in range(start_chunk, max_rows, rows_per_chunk):
        query = QUERY_TEMPLATE.format(limit=rows_per_chunk, offset=offset)
        query_job = client.query(query)
        rows = query_job.result()
        df = rows.to_dataframe()
        df["title_with_body"] = df.title + "\n" + df.body
        yield df

In [ ]:
# Get a dataframe of 1000 rows for demonstration purposes
df = next(query_bigquery_chunks(max_rows=1000, rows_per_chunk=1000))

# Examine the data
df.head()

### Instantiate the text encoding model

Use the [Vertex AI Embeddings for Text API](https://cloud.google.com/vertex-ai/docs/generative-ai/embeddings/get-text-embeddings) developed by Google for converting text to embeddings.

> Text embeddings are a dense vector representation of a piece of content such that, if two pieces of content are semantically similar, their respective embeddings are located near each other in the embedding vector space. This representation can be used to solve common NLP tasks, such as:
> - Semantic search: Search text ranked by semantic similarity.
> - Recommendation: Return items with text attributes similar to the given text.
> - Classification: Return the class of items whose text attributes are similar to the given text.
> - Clustering: Cluster items whose text attributes are similar to the given text.
> - Outlier Detection: Return items where text attributes are least related to the given text.

### Defining an encoding function

Define a function to be used later that will take sentences and convert them to embeddings.

In [ ]:
# Load the "Vertex AI Embeddings for Text" model
model = TextEmbeddingModel.from_pretrained("textembedding-gecko@001")

# Define an embedding method that uses the model
def encode_texts_to_embeddings(sentences: List[str]) -> List[Optional[List[float]]]:
    try:
        embeddings = model.get_embeddings(sentences)
        return [embedding.values for embedding in embeddings]
    except Exception:
        return [None for _ in range(len(sentences))]

### Define two more helper functions for converting text to embeddings

- *generate_batches*: According to the documentation, each request can handle up to 5 text instances. Therefore, this method splits `sentences` into batches of 5 before sending to the embedding API.
- *encode_text_to_embedding_batched*: This method calls `generate_batches` to handle batching and then calls the embedding API via `encode_texts_to_embeddings`. It also handles rate-limiting using `time.sleep`. For production use cases, you would want a more sophisticated rate-limiting mechanism that takes retries into account.

In [ ]:
# Generator function to yield batches of sentences
def generate_batches(
    sentences: List[str], batch_size: int
) -> Generator[List[str], None, None]:
    for i in range(0, len(sentences), batch_size):
        yield sentences[i : i + batch_size]


def encode_text_to_embedding_batched(
    sentences: List[str], api_calls_per_second: int = 10, batch_size: int = 5
) -> Tuple[List[bool], np.ndarray]:

    embeddings_list: List[List[float]] = []

    # Prepare the batches using a generator
    batches = generate_batches(sentences, batch_size)

    seconds_per_job = 1 / api_calls_per_second

    with ThreadPoolExecutor() as executor:
        futures = []
        for batch in tqdm(
            batches, total=math.ceil(len(sentences) / batch_size), position=0
        ):
            futures.append(
                executor.submit(functools.partial(encode_texts_to_embeddings), batch)
            )
            time.sleep(seconds_per_job)

        for future in futures:
            embeddings_list.extend(future.result())

    is_successful = [
        embedding is not None for sentence, embedding in zip(sentences, embeddings_list)
    ]
    embeddings_list_successful = np.squeeze(
        np.stack([embedding for embedding in embeddings_list if embedding is not None])
    )
    return is_successful, embeddings_list_successful

### Test the encoding function

Encode a subset of data and see if the embeddings and distance metrics make sense.

In [ ]:
# Encode a subset of questions for validation
questions = df.title.tolist()[:500]
is_successful, question_embeddings = encode_text_to_embedding_batched(
    sentences=df.title.tolist()[:500]
)

# Filter for successfully embedded sentences
questions = np.array(questions)[is_successful]

Save the dimension size for later usage when creating the index.

In [ ]:
DIMENSIONS = len(question_embeddings[0])

print(DIMENSIONS)

#### Sort questions in order of similarity

Similarity of two embeddings can be calculated using the dot-product between them. 

In this step, you perform the following tasks:

- Calculate vector similarity using `np.dot`
- Sort by similarity scores
- Print results for inspection

Learn more about the [embeddings and semantic search](https://cloud.google.com/vertex-ai/docs/generative-ai/embeddings/get-text-embeddings#colab_example_of_semantic_search_using_embeddings).

In [ ]:
question_index = random.randint(0, 99)

print(f"Query question = {questions[question_index]}")

# Get similarity scores for each embedding by using dot-product.
scores = np.dot(question_embeddings[question_index], question_embeddings.T)

# Print top 20 matches
for index, (question, score) in enumerate(
    sorted(zip(questions, scores), key=lambda x: x[1], reverse=True)[:20]
):
    print(f"\t{index}: {question}: {score}")

### Save the embeddings in JSONL format

The data must be formatted in JSONL format, which means each embedding dictionary is written as an individual JSON object on its own line.

See more information about [input data format and structure](https://cloud.google.com/vertex-ai/docs/matching-engine/match-eng-setup/format-structure#data-file-formats).

In [ ]:
# Create temporary file to write embeddings to
embeddings_file_path = Path(tempfile.mkdtemp())

print(f"Embeddings directory: {embeddings_file_path}")

Write embeddings in batches to prevent out-of-memory errors

In [ ]:
# Set the number of rows needed from the dataset
BQ_NUM_ROWS = 5000
# Set the chunk size for querying and embedding generation
BQ_CHUNK_SIZE = 1000
# Calculate the number of chunks
BQ_NUM_CHUNKS = math.ceil(BQ_NUM_ROWS / BQ_CHUNK_SIZE)
# Initialize the starting chunk index
START_CHUNK = 0

# Create a rate limit of 300 requests per minute. Adjust this depending on your quota.
API_CALLS_PER_SECOND = 300 / 60
# According to the docs, each request can process 5 instances per request
ITEMS_PER_REQUEST = 5

# Loop through each generated dataframe, convert
for i, df in tqdm(
    enumerate(
        query_bigquery_chunks(
            max_rows=BQ_NUM_ROWS, rows_per_chunk=BQ_CHUNK_SIZE, start_chunk=START_CHUNK
        )
    ),
    total=BQ_NUM_CHUNKS - START_CHUNK,
    position=-1,
    desc="Chunk of rows from BigQuery",
):
    # Create a unique output file for each chunk
    chunk_path = embeddings_file_path.joinpath(
        f"{embeddings_file_path.stem}_{i+START_CHUNK}.json"
    )
    with open(chunk_path, "a") as f:
        id_chunk = df.id

        # Convert batch to embeddings
        is_successful, question_chunk_embeddings = encode_text_to_embedding_batched(
            sentences=df.title_with_body,
            api_calls_per_second=API_CALLS_PER_SECOND,
            batch_size=ITEMS_PER_REQUEST,
        )

        # Append to file
        embeddings_formatted = [
            json.dumps(
                {
                    "id": str(id),
                    "embedding": [str(value) for value in embedding],
                }
            )
            + "\n"
            for id, embedding in zip(id_chunk[is_successful], question_chunk_embeddings)
        ]
        f.writelines(embeddings_formatted)

        # Delete the DataFrame and any other large data structures
        del df
        gc.collect()

Upload the training data to a Google Cloud Storage bucket.

In [ ]:
remote_folder = f"{BUCKET_URI}/{embeddings_file_path.stem}/"
! gsutil -m cp -r {embeddings_file_path}/* {remote_folder}

## Create an Index

Configure and create an Index resource in Matching Engine which uses the ANN service (for Production Usage).

Learn more about [ANN service](https://cloud.google.com/vertex-ai/docs/matching-engine/ann-service-overview).

In [ ]:
# Set display name for your index
INDEX_DISPLAY_NAME = "stack_overflow_index_unique" # @param {type:"string"}
# Set description for your index
INDEX_DESCRIPTION = "Query titles and bodies from stackoverflow."

A Matching Engine Index can be created using two methods:
1. `create_tree_ah_index`: Creates an Index resource that uses the asymmetric hashing(AH) algorithm.
2. `create_brute_force_index`: Creates an Index resource that uses the brute force algorithm.

Learn more about the [brute force and AH methods](https://github.com/google-research/google-research/blob/master/scann/docs/algorithms.md).

For a faster perfomance requirement as in production usecases, the AH method is recommended. 

Specify the below parameters to create a Matching Engine Index:

- `display_name`:  The display name of the Index.

- `contents_delta_uri`: Allows inserting, updating  or deleting the contents of the Matching Engine Index. The string must be a valid Google Cloud Storage directory path. 

- `dimensions`: The number of dimensions of the input vectors.

- `approximate_neighbors_count`: The default number of neighbors to find through approximate search before exact reordering is performed. Exact reordering is a procedure where results returned by an approximate search algorithm are reordered via a more expensive distance computation.

- `distance_measure_type`: The distance measure used in nearest neighbor search.

- `leaf_node_embedding_count`: Number of embeddings on each leaf node. The default value is 1000 if not set.

- `leaf_nodes_to_search_percent`: The default percentage of leaf nodes that any query may be searched. Must be in range 1-100, inclusive. The default value is 10 (means 10%) if not set.

- `description`: The description of the Index.  

Learn more about the [parameters to configure Indexes](https://cloud.google.com/vertex-ai/docs/matching-engine/configuring-indexes).

In [ ]:
# create the index
tree_ah_index = aiplatform.MatchingEngineIndex.create_tree_ah_index(
    display_name=INDEX_DISPLAY_NAME,
    contents_delta_uri=remote_folder,
    dimensions=DIMENSIONS,
    approximate_neighbors_count=150,
    distance_measure_type="DOT_PRODUCT_DISTANCE",
    leaf_node_embedding_count=500,
    leaf_nodes_to_search_percent=80,
    description=INDEX_DESCRIPTION,
)
# obtain the Index resource name
INDEX_RESOURCE_NAME = tree_ah_index.resource_name
INDEX_RESOURCE_NAME

Using the resource name, you can retrieve an existing MatchingEngineIndex.

In [ ]:
tree_ah_index = aiplatform.MatchingEngineIndex(index_name=INDEX_RESOURCE_NAME)

## Create an IndexEndpoint

To query and use your created Index, you need to deploy your Index to an IndexEndpoint. This concept is similar to how Vertex AI Models need to be deployed to an Endpoint for online predictions.

Sepcify the following parameter sto create an IndexEndpoint:

- `display_name`: The display name of the IndexEndpoint.
- `description`: The description of the IndexEndpoint.
- `public_endpoint_enabled`: If true, the deployed index will be accessible through public endpoint.

Learn more about [creating IndexEndpoints](https://cloud.google.com/vertex-ai/docs/matching-engine/deploy-index-public#create-index-endpoint).

In [ ]:
# Set the display name
ENDPOINT_DISPLAY_NAME = "stack_overflow_endpoint_unique" # @param {type:"string"}
ENDPOINT_DESCRIPTION = "IndexEndpoint to receive the query requests for stackoverflow."

my_index_endpoint = aiplatform.MatchingEngineIndexEndpoint.create(
    display_name=DISPLAY_NAME,
    description=DISPLAY_NAME,
    public_endpoint_enabled=True,
)

## Deploy the Index

Set an id for your deployment. 

In [ ]:
# Set deployment id
DEPLOYED_INDEX_ID = "stack_overflow_deployment_unique" # @param {type:"string"}

DEPLOYED_INDEX_ID

Deploy your Index to the IndexEndpoint.

In [ ]:
# Deploy the Index
my_index_endpoint = my_index_endpoint.deploy_index(
    index=tree_ah_index, 
    deployed_index_id=DEPLOYED_INDEX_ID
)

my_index_endpoint.deployed_indexes

### Verify number of declared items matches the number of embeddings

Each IndexEndpoint can have multiple indexes deployed to it. For each index, you can retrieved the number of deployed vectors using the `index_endpoint._gca_resource.index_stats.vectors_count`. The numbers may not match exactly due to potential failures using the embedding service.

In [ ]:
number_of_vectors = sum(
    aiplatform.MatchingEngineIndex(
        deployed_index.index
    )._gca_resource.index_stats.vectors_count
    for deployed_index in my_index_endpoint.deployed_indexes
)

print(f"Expected: {BQ_NUM_ROWS}, Actual: {number_of_vectors}")

## Create online queries

Specify your query and encode it as an embedding.

In [ ]:
# Specify your query(s)
QUERY_LIST = ["Install GPU for Tensorflow"]
# Encode your query
test_embeddings = encode_texts_to_embeddings(sentences=QUERY_LIST)

After you build your Index, you may query against the deployed Index to find nearest neighbors.

Note: For the **DOT_PRODUCT_DISTANCE** distance type, the "distance" property returned with each MatchNeighbor object actually refers to the similarity score.

In [ ]:
# Set number of neighbors to return
NUM_NEIGHBOURS = 10
# Query the deployed index
response = my_index_endpoint.find_neighbors(
    deployed_index_id=DEPLOYED_INDEX_ID,
    queries=test_embeddings,
    num_neighbors=NUM_NEIGHBOURS,
)
# show response
response

Verify that the retrieved results are relevant by checking the StackOverflow links.

In [ ]:
for match_index, neighbor in enumerate(response[0]):
    print(f"https://stackoverflow.com/questions/{neighbor.id}")

## Storing and retrieving titles from a Redis data store

When you productionize this code into a service, you need to convert the nearest IDs returned from Vertex AI Matching Engine into data ready to be used by downstream services.

In this case, you need to convert the IDs back to titles. You can use Google Cloud's Memorystore to deploy a managed Redis instance to save the id-title key-value pairs.

To learn more, see [Create and manage Redis instances](https://cloud.google.com/memorystore/docs/redis/create-manage-instances?hl=en).

**Note:** Below, you create a Redis instance with the default `DIRECT_PEERING` mode. Learn more about the [modes available for Redis instances](https://cloud.google.com/memorystore/docs/redis/networking?hl=en#connection_modes).

In [ ]:
# Set the Redis instance name
REDIS_INSTANCE_NAME = "stackoverflow-questions-unique" # @param {type:"string"}

# Create a Redis instance
! gcloud redis instances create '{REDIS_INSTANCE_NAME}' --size=10 --region='{REGION}'

Obtain the host address and port information for the created Redis instance.

In [ ]:
# Get host and port info
REDIS_HOST = ! gcloud redis instances list --filter="INSTANCE_NAME:'{REDIS_INSTANCE_NAME}'" --region {REGION}  --format='value(HOST)'
REDIS_PORT = ! gcloud redis instances list --filter="INSTANCE_NAME:'{REDIS_INSTANCE_NAME}'" --region {REGION} --format='value(PORT)'

if isinstance(REDIS_HOST, list):
    REDIS_HOST = REDIS_HOST[0]

if isinstance(REDIS_PORT, list):
    REDIS_PORT = REDIS_PORT[0]

print(f"REDIS_HOST = {REDIS_HOST}")
print(f"REDIS_PORT = {REDIS_PORT}")

Connect to the Redis instance as a client using the Redis SDK for Python.

In [ ]:
# Connect to the instance
redis_client = redis.StrictRedis(host=REDIS_HOST, port=REDIS_PORT)

Create a Redis pipeline and set the specified fields (IDs) to their respective values(Title, Body) in the hash stored at key.

In [ ]:
%%time
# Convert the id -> (title, body) relationship into a dict and write to Redis
for df in tqdm(
    query_bigquery_chunks(
        max_rows=BQ_NUM_ROWS, rows_per_chunk=BQ_CHUNK_SIZE, start_chunk=0
    ),
    total=BQ_NUM_CHUNKS,
    position=0,
    desc="Chunk of rows from BigQuery",
):
    ids = df.id.tolist()
    titles = df.title.tolist()
    bodies = df.body.tolist()

    # create a Redis pipeline
    pipe = redis_client.pipeline()

    # iterate over the data and add hset commands to the pipeline
    for (id, title, body) in tqdm(zip(ids, titles, bodies), total=len(ids), position=1):
        pipe.hset(
            str(id),
            mapping={
                "title": str(title),
                "body": str(body[:100]),
            },
        )

    # execute the pipeline
    _ = pipe.execute()

Select 10 IDs at random from the dataset for querying and verify the responses from the Redis instance.

In [ ]:
# Verify that Redis can retrieve the correct information
df = df.sample(10)

[
    f"Actual = {title}, Retrieved = {redis_client.hgetall(str(id))}"
    for id, title in zip(df.id, df.title)
]

## Cleaning up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial:

- IndexEndpoint
- Index
- Redis instance
- Cloud Storage bucket (set `delete_bucket` to **True** for deletion)

In [ ]:
# Force undeployment of indexes and delete endpoint
my_index_endpoint.delete(force=True)

# Delete indexes
tree_ah_index.delete()

# Delete redis instance
! gcloud redis instances delete '{REDIS_INSTANCE_NAME}' --region {REGION} --quiet

# Delete Cloud Storage objects that were created
delete_bucket = False
if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil -m rm -r $BUCKET_URI